In [ ]:
# ==============================================================================
# Step 1: Install Dependencies
# ==============================================================================
# It's a good practice to include a cell for installing required libraries.
# This ensures that the notebook is self-contained and reproducible.
!pip install replicate langchain gradio langchain_community langchain_huggingface faiss-cpu transformers python-dotenv

# ==============================================================================
# Step 2: Securely Load Environment Variables
# ==============================================================================
# This is the corrected and secure way to handle API keys.
# ⚠️ Instructions for the user:
# Before running, create a file named ".env" in the same directory as this notebook.
# Add your Replicate token to the file like this:
# REPLICATE_API_TOKEN="your_token_here"
# Make sure to add this ".env" file to your .gitignore so it is not uploaded to GitHub.
#
# If running on Google Colab, use the "Secrets" feature instead.
# In that case, use `os.environ["REPLICATE_API_TOKEN"] = userdata.get("REPLICATE_API_TOKEN")`

import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Check if the API token is loaded
replicate_api_token = os.getenv("REPLICATE_API_TOKEN")
if not replicate_api_token:
    raise ValueError("REPLICATE_API_TOKEN not found. Please set it in your .env file or Colab Secrets.")

# ==============================================================================
# Step 3: Core Logic - Setup LLM, Vector Store, and RAG Chain
# ==============================================================================
import gradio as gr
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Replicate
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from transformers import AutoTokenizer
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_community.embeddings import HuggingFaceEmbeddings

# IBM Granite model via Replicate
model_path = "ibm-granite/granite-3.3-8b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = Replicate(
    model=model_path, 
    replicate_api_token=replicate_api_token, # Use the securely loaded token
    input={"temperature": 0.1, "max_length": 500}
)

# 📚 Sample LearnMate knowledge base
filename = "learnmate_courses.txt"
with open(filename, "w") as f:
    f.write("""
Frontend Development Pathway:
1. HTML & CSS Fundamentals (Beginner) — Learn basic web page structure and styling.
2. JavaScript Essentials (Beginner) — Add interactivity to websites.
3. React for Developers (Intermediate) — Build dynamic, modern web apps.
4. Frontend Capstone Project (Advanced) — Create a real-world portfolio project.

Cybersecurity Pathway:
1. Intro to Cybersecurity (Beginner) — Understand basic security concepts.
2. Network Security (Intermediate) — Learn to secure communication channels.
3. Ethical Hacking (Advanced) — Test systems for vulnerabilities ethically.

UI/UX Design Pathway:
1. Intro to UI/UX (Beginner) — Learn the basics of user interface and experience.
2. Wireframing & Prototyping (Intermediate) — Create interactive prototypes.
3. UI/UX Capstone (Advanced) — Complete a design project for a real client.

Tips for Learning:
- Always build projects alongside learning theory.
- Review and adapt your learning roadmap every 2 months.
- Start from your current skill level and progress gradually.
""")

# 📄 Load & Split content
loader = TextLoader(filename)
documents = loader.load()
splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=50)
texts = splitter.split_documents(documents)

# 🔍 Vector store (FAISS for quick Colab testing)
# The `HuggingFaceEmbeddings` import is correct and will use the model locally
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_db = FAISS.from_documents(texts, embeddings)

# 🎯 Prompt for LearnMate
template = """
You are LearnMate, an AI learning coach.
User Question: {question}
Based on the student's interests, skill level, and goals, give a personalized learning pathway.
Suggest specific courses, explain why they fit, and provide tips for progression.
"""
prompt = PromptTemplate(template=template, input_variables=["question"])

# 🔗 Chains
llm_chain = LLMChain(llm=model, prompt=prompt)
combine_chain = StuffDocumentsChain(llm_chain=llm_chain)

rag_chain = RetrievalQA(
    retriever=vector_db.as_retriever(),
    combine_documents_chain=combine_chain,
    return_source_documents=False
)

# ==============================================================================
# Step 4: Gradio Interface
# ==============================================================================
def ask_learnmate(query):
    """
    Function to handle the Gradio input and generate a response.
    """
    if not query:
        return "Please enter a question."

    try:
        response = rag_chain.run(query)
        return response
    except Exception as e:
        return f"❌ An error occurred: {str(e)}"

iface = gr.Interface(
    fn=ask_learnmate,
    inputs=gr.Textbox(label="Ask LearnMate", placeholder="e.g. How do I become a frontend developer?"),
    outputs=gr.Textbox(label="LearnMate's Advice"),
    title="LearnMate — Personalized Course Pathways",
    description="Get a personalized learning roadmap for your career goals in frontend development, cybersecurity, or UI/UX.",
    theme="default"
)

# Launch the interface
iface.launch(share=True)